In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TEK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [10]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0.1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [11]:
list(zip(longforms, counts))

[('transposable element silencing via at hook', 10),
 ('traditional ecological knowledge', 7),
 ('tyrosine kinase endothelial', 2),
 ('tek tyrosine kinase', 1),
 ('tyrosine kinase receptor', 1),
 ('kdr and tie2', 1),
 ('tie2 one of the tyrosine kinase', 1),
 ('vegfa kdr flt1 and angiopoietin1', 1),
 ('tapetal ahl family protein', 1),
 ('tbk1 and a pcg causing gene', 1),
 ('the angiopoietin 1 receptor', 1),
 ('members of the ahl family including ahl16', 1)]

In [16]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [17]:
result = [grounding_map, names, pos_labels]

In [18]:
result

[{'kdr and tie2': 'HGNC:12701',
  'members of the ahl family including ahl16': 'UP:Q9SJG4',
  'tapetal ahl family protein': 'UP:Q9SJG4',
  'tbk1 and a pcg causing gene': 'HGNC:12701',
  'tek tyrosine kinase': 'HGNC:12701',
  'the angiopoietin 1 receptor': 'HGNC:12701',
  'tie2 one of the tyrosine kinase': 'HGNC:12701',
  'traditional ecological knowledge': 'ungrounded',
  'transposable element silencing via at hook': 'UP:Q9SJG4',
  'tyrosine kinase endothelial': 'HGNC:12701',
  'tyrosine kinase receptor': 'HGNC:12701',
  'vegfa kdr flt1 and angiopoietin1': 'HGNC:12701'},
 {'HGNC:12701': 'TEK', 'UP:Q9SJG4': 'AHL16'},
 ['HGNC:12701', 'UP:Q9SJG4']]

In [20]:
grounding_map, names, pos_labels = [{'tie2': 'HGNC:12701',
  'ahl16': 'UP:Q9SJG4',
  'tapetal ahl family protein': 'UP:Q9SJG4',
  'tek tyrosine kinase': 'HGNC:12701',
  'angiopoietin 1 receptor': 'HGNC:12701',
  'tie2 one of the tyrosine kinase': 'HGNC:12701',
  'traditional ecological knowledge': 'ungrounded',
  'transposable element silencing via at hook': 'UP:Q9SJG4',
  'tyrosine kinase endothelial': 'HGNC:12701',
  'tyrosine kinase receptor': 'HGNC:12701'},
 {'HGNC:12701': 'TEK', 'UP:Q9SJG4': 'AHL16'},
 ['HGNC:12701', 'UP:Q9SJG4']]

In [21]:
excluded_longforms = []

In [22]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [23]:
additional_entities = {}

In [24]:
unambiguous_agent_texts = {}

In [25]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [26]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [27]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [28]:
intersection1

[]

In [29]:
intersection2

[]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [30]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 03:57:20] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 03:57:22] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8666666666666666 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [31]:
classifier.stats

{'label_distribution': {'HGNC:12701': 5, 'ungrounded': 3, 'UP:Q9SJG4': 8},
 'f1': {'mean': 0.866667, 'std': 0.266667},
 'precision': {'mean': 0.85, 'std': 0.3},
 'recall': {'mean': 0.9, 'std': 0.2},
 'ungrounded': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:12701': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'UP:Q9SJG4': {'f1': {'mean': 0.933333, 'std': 0.133333},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.9, 'std': 0.2}}}

In [32]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [33]:
disamb.dump(model_name, results_path)

In [34]:
print(disamb.info())

Disambiguation model for TEK

Produces the disambiguations:
	AHL16*	UP:Q9SJG4
	TEK*	HGNC:12701

Class level metrics:
--------------------
Grounding	Count	F1     
AHL16*	8	0.93333
  TEK*	5	    0.8
Ungrounded	3	    0.6

Weighted Metrics:
-----------------
	F1 score:	0.86667
	Precision:	0.85
	Recall:		0.9

* Positive labels
See Docstring for explanation



In [35]:
model_to_s3(disamb)